In [1]:
import time
import cv2 
import os
import mediapipe as mp  
import HandTrackingModule as htm

In [3]:
cap = cv2.VideoCapture(0)

w_cam = 640
h_cam = 480

cap.set(3, w_cam) #sets width
cap.set(4, h_cam) #sets height

folderPath = "images/num_finger"
myList = os.listdir(folderPath)
# print(myList) 
overlayList = []


for imPath in myList:
  image = cv2.imread(f'{folderPath}/{imPath}')
  overlayList.append(image) #to save the imported image

# print(len(overlayList))       #output = 6

pTime = 0

detector=htm.handDetector(detectionCon = 0.75)


while True:
  success, img = cap.read()
  
  img = cv2.flip(img, 1)
  
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = detector.findHands(img)
  
  lmList = detector.findPosition(img, draw=False)
  tipIds = [4, 8, 12, 16, 20]
  
  if len(lmList) != 0:
    fingers = []
    for id in range(0,5):
      if id != 0: #fingers
        if lmList[tipIds[id]][2]<lmList[tipIds[id]-2][2]:
          fingers.append(1)
        else:
          fingers.append(0)
          
      if id == 0: #thumb
        if lmList[tipIds[id]][1]<lmList[tipIds[id]-1][1]:
          fingers.append(1)
        else:
          fingers.append(0)
      

    # print(fingers.count(1))
    
    h, w, c = overlayList[fingers.count(1)].shape
    img[0:h, 0:w] = overlayList[fingers.count(1)]
    
    cv2.rectangle(img,(5,h+5),(30,h+65),(0,255,0),cv2.FILLED)
    cv2.putText(img, f"{fingers.count(1)}", (11,h+49), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,0),3)
  
  
  cTime= time.time()
  fps=1/(cTime-pTime)
  pTime = cTime
  
  cv2.putText(img, f'FPS={int(fps)}', (40,h_cam - 50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
  cv2.putText(img, f"Use Right hand's Light Side", (40,h_cam - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,0),3)

  
  
  cv2.imshow("Finger Counter", img)
  
  cv2.waitKey(1)
  if cv2.waitKey(1) & 0xFF == ord('q'): 
    break  
   
cap.release()
cv2.destroyAllWindows()